## Twitter Data Analysis
@author Swati Singh

Github: https://github.com/swatisingh0107

Note: For code updates, visit my Github profile. The below mentioned changes will be published on Github page in future.
### Remaining Things to do
1. Extract hashtags from text column and store in a separate column in dataframe
2. Fetch wordcloud for most discussed brandnames.
3. Clean up text column to remove hyperlinks, hashtags and mentions.

First we are going to load the Twitter JSOn file into a variable JSONdata

In [1]:
import os
os.getcwd()

'C:\\Users\\swati\\Google Drive\\Superbowl'

In [2]:
import json
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
JSONdata='C:\\Users\\swati\\Google Drive\\Superbowl\\Dataset\\SuperbowlJS.json'


### Read JSON data to a list
Now we will read twitter JSON file into a list 'tweets'. Please note Twitter JSON is encoded using UTF-8 characters. Hence the parser should be able to decode the JSON file accurately.

In [4]:
tweets = []
with open(JSONdata,'r',encoding="utf8") as f:
    tweets = [json.loads(line) for line in f]

Print text in first row.

In [5]:
 tweets[1]['text']    

'RT @IAStartingLine: Bud Light has now finished last in the Iowa Caucus https://t.co/hUUjmX62to'

### Total number of records

In [6]:
len(tweets)

859485

### Understanding JSON structure of tweet data
Now we will look at the structure of the JSON to look for relevant keys that we will use for analysis.

In [7]:
import json
tweet = tweets[6]
print(json.dumps(tweet, indent=4))

{
    "_id": {
        "$oid": "5c57b5f92cec0f358301cf6d"
    },
    "quote_count": 0,
    "contributors": null,
    "truncated": false,
    "text": "RT @GameOfThrones: .@TheBudKnight\u2019s watch has ended.\nWhat will you do #ForTheThrone?\n#SBLIII @BudLight https://t.co/SzhvDcNAaL",
    "is_quote_status": false,
    "in_reply_to_status_id": null,
    "reply_count": 0,
    "id": {
        "$numberLong": "1092261582329995269"
    },
    "favorite_count": 0,
    "entities": {
        "user_mentions": [
            {
                "id": 180463340,
                "indices": [
                    3.0,
                    17.0
                ],
                "id_str": "180463340",
                "screen_name": "GameOfThrones",
                "name": "Game Of Thrones"
            },
            {
                "id": {
                    "$numberLong": "942879856160800768"
                },
                "indices": [
                    20.0,
                    33.0
           

### Example of List item

In [31]:
tweet['source']

'<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>'

### Extract retweeted handles.
The JSON does not reflect the retweeted count as expected. Hence we will use the information from the text to flag tweets that were retweeted.

In [9]:
for tweet in tweets:
    if 'RT @' in tweet['text']: 
        tweet['retweeted_from'] = tweet['text'].split(':',1)[0]
#         tweet['text']= tweet['text'].split(':',1)[1]
    else:    
        tweet['retweeted_from'] = 'other'

### Transform tweets list to a dataframe. 
Exclude key:value pairs that are not required for our analysis.

In [10]:

column_list=['created_at','id_str','text','retweeted_from','source','retweet_count','favorite_count',
             'screen_name','followers_count','longitude','latitude']

tweet_data = []
for tweet in tweets:
    tweet_data.append([tweet[key] for key in column_list[:-4]])
    tweet_data[-1].append(tweet['user']['screen_name'])
    tweet_data[-1].append(tweet['user']['followers_count'])
#     if 'extended_tweet' in tweet.keys():
#         tweet_data[-1].append(tweet['extended_tweet']['full_text'])
#     else:
#         tweet_data[-1].append("None")
            
    if 'derived' in tweet['user'].keys():
        tweet_data[-1].append(tweet['user']['derived']['locations'][0]['geo']['coordinates'][0])
        tweet_data[-1].append(tweet['user']['derived']['locations'][0]['geo']['coordinates'][1])
    
    
data = pd.DataFrame(tweet_data, columns=column_list)
data.head()

,created_at,id_str,text,retweeted_from,source,retweet_count,favorite_count,screen_name,followers_count,longitude,latitude
0,Mon Feb 04 03:20:19 +0000 2019,1092261543859884032,Best commercial BY FAR was #NFL100 ad. Rest we...,other,"<a href=""http://twitter.com/download/iphone"" r...",0,0,MelanieG_Writer,1031,-96.66696,40.80000
1,Mon Feb 04 03:20:19 +0000 2019,1092261541137780736,RT @IAStartingLine: Bud Light has now finished...,RT @IAStartingLine,"<a href=""http://twitter.com/download/iphone"" r...",0,0,CaitrinSneed,456,-87.65005,41.85003
2,Mon Feb 04 03:20:21 +0000 2019,1092261550738599942,RT @GameOfThrones: .@TheBudKnight’s watch has ...,RT @GameOfThrones,"<a href=""http://twitter.com/download/android"" ...",0,0,tpalomaresc,72,NaN,NaN
3,Mon Feb 04 03:20:22 +0000 2019,1092261555008401409,RT @GameOfThrones: .@TheBudKnight’s watch has ...,RT @GameOfThrones,"<a href=""http://twitter.com/download/iphone"" r...",0,0,haileyncrawford,104,NaN,NaN
4,Mon Feb 04 03:20:25 +0000 2019,1092261568404967425,RT @GameOfThrones: .@TheBudKnight’s watch has ...,RT @GameOfThrones,"<a href=""http://twitter.com/download/android"" ...",0,0,jamesmortlock1,14,NaN,NaN


### Drop duplicated records

In [11]:
data=data.drop_duplicates('id_str')

### Total records to be used for analysis

In [12]:
len(data)

767453

### Clean source columns to remove HTML format. Also clean retweeted_from column to remove redundant characters.

In [13]:
data['source'] = data['source'].str.split('>').str.get(1)
data['source']=data['source'].str.replace('</a','')
data['retweeted_from']=data['retweeted_from'].str.replace('RT @','')

### Transform created_at column from string to date-time format

In [14]:
data['created_at']=pd.to_datetime(data['created_at'])

### Keep records captured only during game time

In [15]:
data=data[(data['created_at']>'2019-02-03 23:29:00') & (data['created_at']<'2019-02-04 04:00:00')]
data.head()

,created_at,id_str,text,retweeted_from,source,retweet_count,favorite_count,screen_name,followers_count,longitude,latitude
0,2019-02-04 03:20:19,1092261543859884032,Best commercial BY FAR was #NFL100 ad. Rest we...,other,Twitter for iPhone,0,0,MelanieG_Writer,1031,-96.66696,40.80000
1,2019-02-04 03:20:19,1092261541137780736,RT @IAStartingLine: Bud Light has now finished...,IAStartingLine,Twitter for iPhone,0,0,CaitrinSneed,456,-87.65005,41.85003
2,2019-02-04 03:20:21,1092261550738599942,RT @GameOfThrones: .@TheBudKnight’s watch has ...,GameOfThrones,Twitter for Android,0,0,tpalomaresc,72,NaN,NaN
3,2019-02-04 03:20:22,1092261555008401409,RT @GameOfThrones: .@TheBudKnight’s watch has ...,GameOfThrones,Twitter for iPhone,0,0,haileyncrawford,104,NaN,NaN
4,2019-02-04 03:20:25,1092261568404967425,RT @GameOfThrones: .@TheBudKnight’s watch has ...,GameOfThrones,Twitter for Android,0,0,jamesmortlock1,14,NaN,NaN


In [16]:
len(data)

753102

### Count number of retweets

In [17]:
retweet_count=data.groupby(['retweeted_from','text']).size().reset_index(name='counts')

In [21]:
import numpy as np
# count_stat=data.groupby(['retweeted_from'])[["favorite_count","retweet_count"]].sum()
count_stat=data.groupby(['retweeted_from']).agg({'favorite_count':np.sum,'retweet_count':np.sum})
# df.groupby(['Country', 'Item_Code']).agg({'Y1961': np.sum, 'Y1962': [np.sum, np.mean]})  # Added example for two output columns from a single input column
count_stat.head()
#Looks lieke this data is not captured accurately to derive any meaningful information. we will use the next piece 
#of code to count the most retweeted tweet and check most popular brand account


,favorite_count,retweet_count
retweeted_from,,
!!! CNET,0,0
#BVSBooks,1,1
#ElvisDuranShow,0,0
#EverythingWeGot #gopats theatlantavoice,0,0
#LARams #gorams QuarterPress,0,0


In [22]:
retweet_count.sort_values(['counts'],ascending=False).head()

,retweeted_from,text,counts
1797,Avengers,"RT @Avengers: ""Some people move on. But not us...",54248
13062,MarvelStudios,RT @MarvelStudios: HIGHER. FURTHER. FASTER. Wa...,12944
7683,GameOfThrones,RT @GameOfThrones: .@TheBudKnight’s watch has ...,9837
24999,captainmarvel,RT @captainmarvel: HIGHER. FURTHER. FASTER. Wa...,7155
604,AVG_JP,RT @AVG_JP: 人々はもう\n 諦めている\n我々は違う\n 始めるぞ\n\n『ア...,6712


### To do: To transform this into a function to return most and least popular tweet

In [23]:
RT_max=np.max(retweet_count['counts'])
rt=retweet_count[retweet_count.counts==RT_max].index[0]
print("The tweet with more retweets is: \n{}".format(retweet_count['text'][rt]))
print("Number of retweets: {}".format(RT_max))


The tweet with more retweets is: 
RT @Avengers: "Some people move on. But not us." Watch the brand new Marvel Studios' #AvengersEndgame spot that aired during the Big Game.…
Number of retweets: 54248


### Most popular twitter sources

In [24]:
source_count=data.groupby(['source']).size().reset_index(name='counts')
source_count.sort_values(['counts'],ascending=False).head()

,source,counts
1090,Twitter for iPhone,413380
1086,Twitter for Android,201656
1083,Twitter Web Client,58065
1082,Twitter Web App,27201
1089,Twitter for iPad,13673


### Sentiment Analysis

In [25]:
from textblob import TextBlob
import re

def clean_tweet(tweet):
    '''
    Utility function to clean the text in a tweet by removing 
    links and special characters using regex.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def analize_sentiment(tweet):
    '''
    Utility function to classify the polarity of a tweet
    using textblob.
    '''
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [26]:
data['SA'] = data['text'].apply(analize_sentiment)

In [27]:
data.head()

,created_at,id_str,text,retweeted_from,source,retweet_count,favorite_count,screen_name,followers_count,longitude,latitude,SA
0,2019-02-04 03:20:19,1092261543859884032,Best commercial BY FAR was #NFL100 ad. Rest we...,other,Twitter for iPhone,0,0,MelanieG_Writer,1031,-96.66696,40.80000,Positive
1,2019-02-04 03:20:19,1092261541137780736,RT @IAStartingLine: Bud Light has now finished...,IAStartingLine,Twitter for iPhone,0,0,CaitrinSneed,456,-87.65005,41.85003,Positive
2,2019-02-04 03:20:21,1092261550738599942,RT @GameOfThrones: .@TheBudKnight’s watch has ...,GameOfThrones,Twitter for Android,0,0,tpalomaresc,72,NaN,NaN,Neutral
3,2019-02-04 03:20:22,1092261555008401409,RT @GameOfThrones: .@TheBudKnight’s watch has ...,GameOfThrones,Twitter for iPhone,0,0,haileyncrawford,104,NaN,NaN,Neutral
4,2019-02-04 03:20:25,1092261568404967425,RT @GameOfThrones: .@TheBudKnight’s watch has ...,GameOfThrones,Twitter for Android,0,0,jamesmortlock1,14,NaN,NaN,Neutral


### Count tweets and handles based on positive and negative sentiment

In [44]:
#For retweet analysis, we will remove rows where retweeted from is others. These are rows which are actual tweets and not retweets.
retweet = data[data['retweeted_from']!='other']
len(retweet)

470984

In [59]:
sentiment_count=retweet.groupby(['retweeted_from','text','SA']).size().reset_index(name='counts')

sentiment_count[sentiment_count['counts']>2000].sort_values(['SA','counts'],ascending=False)

,retweeted_from,text,SA,counts
13062,MarvelStudios,RT @MarvelStudios: HIGHER. FURTHER. FASTER. Wa...,Positive,12944
24999,captainmarvel,RT @captainmarvel: HIGHER. FURTHER. FASTER. Wa...,Positive,7155
8900,IGN,RT @IGN: 🚨 NEW AVENGERS: ENDGAME FOOTAGE 🚨 htt...,Positive,3998
13028,Marvel,RT @Marvel: HIGHER. FURTHER. FASTER. Watch the...,Positive,3912
16033,PatrickMahomes5,RT @PatrickMahomes5: RT &amp; @Verizon will do...,Positive,3535
13611,Microsoft,RT @Microsoft: Just aired: Our #SuperBowl comm...,Positive,3267
22435,_malikab_,RT @_malikab_: #PepsiHalftime retweet for swee...,Positive,2463
2742,Bijansiry,RT @Bijansiry: Trying to prove a point\nRT for...,Positive,2261
11598,LakeShowYo,RT @LakeShowYo: that Super Bowl #PepsiHalftime...,Positive,2056
13514,Micboom008,RT @Micboom008: When they only referenced Spon...,Positive,2056


In [66]:
def tweet_sentiment(sentiment):
    sentiment_data=sentiment_count[(sentiment_count['SA']==sentiment)& (sentiment_count['retweeted_from']!='other')]
    max_sentiment=np.max(sentiment_data['counts'])
    sentiment_index=sentiment_count[sentiment_count.counts==max_sentiment].index[0]
    print("The retweet with most "+ sentiment + " sentiment is: \n{}".format(sentiment_count['text'][sentiment_index])+" \nfrom "+(sentiment_count['retweeted_from'][sentiment_index])+ "\nCount="+str(sentiment_count['counts'][sentiment_index]))
    
tweet_sentiment('Positive')    
print("-----------------------------------------------------------------")
tweet_sentiment('Neutral')
print("-----------------------------------------------------------------")
tweet_sentiment('Negative')

The retweet with most Positive sentiment is: 
RT @MarvelStudios: HIGHER. FURTHER. FASTER. Watch the brand-new #CaptainMarvel spot that aired during the big game. See it in theaters Marc… 
from MarvelStudios
Count=12944
-----------------------------------------------------------------
The retweet with most Neutral sentiment is: 
RT @GameOfThrones: .@TheBudKnight’s watch has ended.
What will you do #ForTheThrone?
#SBLIII @BudLight https://t.co/SzhvDcNAaL 
from GameOfThrones
Count=9837
-----------------------------------------------------------------
The retweet with most Negative sentiment is: 
RT @Avengers: "Some people move on. But not us." Watch the brand new Marvel Studios' #AvengersEndgame spot that aired during the Big Game.… 
from Avengers
Count=54248
